### 1. Kütüphanelerin İçe Aktarılması
 numpy, pandas: Veri işleme

torch: Derin öğrenme modeli

sklearn: Veri ön işleme ve değerlendirme metrikleri

LabelEncoder: Hedef değişkeni (Action) sayısal hale getirmek için

In [1]:
# ===============================
# Gerekli Kütüphaneler
# ===============================
import os
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight



### 2. Genel Ayarlar (Tekrar Üretilebilirlik)

Aynı sonuçları elde edebilmek için sabit bir seed kullanılır

GPU varsa otomatik olarak kullanılır

In [2]:
# ===============================
# Genel Ayarlar
# ===============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


### 3. Veri Setinin Yüklenmesi

Firewall veri seti CSV dosyasından okunur

İlk birkaç satır kontrol edilir

In [3]:
# ===============================
# Veri Setinin Yüklenmesi
# ===============================
DATA_PATH = "log2.csv"
df = pd.read_csv(DATA_PATH)

df.head()


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


### 4. Veri Ön İşleme (Preprocessing)

Action sütunu modelin tahmin edeceği sınıftır

StandardScaler özelliklerin aynı ölçeğe getirilmesini sağlar

stratify=y sınıf dağılımını korur

In [4]:
# ===============================
# Veri Ön İşleme
# ===============================
TARGET_COL = "Action"

FEATURE_COLS = [
    "Source Port", "Destination Port", "NAT Source Port",
    "NAT Destination Port", "Bytes", "Bytes Sent", "Bytes Received",
    "Packets", "Elapsed Time (sec)", "pkts_sent", "pkts_received"
]

# Girdi ve hedef değişkenlerin ayrılması
X = df[FEATURE_COLS].values

# Hedef değişken string olduğu için sayısal hale getirilir
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[TARGET_COL])

# Özelliklerin ölçeklenmesi
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Eğitim ve test verisinin ayrılması
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)


### 5. PyTorch Dataset ve DataLoader

Veri PyTorch’un anlayacağı formata dönüştürülür

DataLoader eğitim sırasında veriyi parça parça (batch) verir

In [5]:
# ===============================
# PyTorch Dataset Tanımı
# ===============================
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TabularDataset(X_train, y_train)
test_dataset  = TabularDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=512, shuffle=False)
    

### 6. Yapay Sinir Ağı Modeli (MLP)

Model çok katmanlı ileri beslemeli sinir ağıdır

Dropout aşırı öğrenmeyi (overfitting) önler

Çıkış katmanı sınıf sayısı kadar nöron içerir

In [6]:
# ===============================
# Çok Katmanlı Algılayıcı (MLP)
# ===============================
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),

            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)


model = MLP(input_dim=X.shape[1], num_classes=len(np.unique(y)))
model.to(DEVICE)


MLP(
  (net): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=4, bias=True)
  )
)

### 7. Class Weight ve Kayıp Fonksiyonu

Veri seti dengesiz olduğu için sınıf ağırlıkları kullanılır

Bu sayede nadir sınıflar göz ardı edilmez

In [7]:
# ===============================
# Genel Ayarlar
# ===============================

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)


### 8. Modelin Eğitilmesi


Her epoch’ta tüm eğitim verisi kullanılır

Geri yayılım (backpropagation) ile ağırlıklar güncellenir

In [ ]:
# ===============================
# Eğitim Döngüsü
# ===============================
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for Xb, yb in train_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {total_loss/len(train_loader):.4f}")


### 9. Modelin Değerlendirilmesi

Test verisi ile model performansı ölçülür

confusion_matrix: sınıflar arası hataları gösterir

classification_report: precision, recall ve F1-score değerlerini verir

In [ ]:
# ===============================
# Model Değerlendirme
# ===============================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb = Xb.to(DEVICE)
        outputs = model(Xb)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        y_true.extend(yb.numpy())
        y_pred.extend(preds)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))
